In [1]:
#!pip install backtrader

In [2]:
from datetime import datetime
import backtrader as bt
import matplotlib

matplotlib.__version__

'3.2.2'

In [3]:
cerebro = bt.Cerebro()

In [4]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        # To keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.MACDHisto(self.datas[0])
        bt.indicators.ATR(self.datas[0], plot=False)

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f"{dt}, {txt}")
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm {order.executed.comm}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm 
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm {order.executed.comm}')
    
            

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset order status to prepare for next order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}')
        
    def stop(self):
        #self.log(f"(MA Period {self.params.maperiod}) Ending Value {round(self.broker.getvalue(),2)}", doprint=True)
        print(f"\n(MA Period {self.params.maperiod}) Ending Value {round(self.broker.getvalue(),2)}")


    def next(self):
        # Simply log the closing price of the series from the reference
        self.log(f'Close, {self.dataclose[0]}')

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log(f'BUY CREATE, {self.dataclose[0]}')

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log(f'SELL CREATE, {self.dataclose[0]}')

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [5]:
# Add a strategy
cerebro.optstrategy(TestStrategy, maperiod=range(8, 34+1))

In [6]:
data = bt.feeds.YahooFinanceData(dataname='^NSEI', fromdate=datetime(2020, 1, 1),
                                  todate=datetime(2021, 2, 16))
cerebro.adddata(data)

In [7]:
cerebro.broker.setcash(10_00_000.0)

In [8]:
# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=3) #buy/sell 3 futures

In [9]:
# Set the commission - Rs 20 per transcation
cerebro.broker.setcommission(commission=20, margin=1_70_000)

In [10]:
starting = cerebro.broker.getvalue()
print(f'Starting Portfolio Value: {starting}')

Starting Portfolio Value: 1000000.0


In [11]:
%%time
cerebro.run()


(MA Period 13) Ending Value 1010442.4

(MA Period 10) Ending Value 1009893.1

(MA Period 9) Ending Value 1009253.35
(MA Period 12) Ending Value 1011641.05


(MA Period 11) Ending Value 1011434.8

(MA Period 8) Ending Value 1008303.1

(MA Period 14) Ending Value 1011534.4

(MA Period 15) Ending Value 1012678.6
(MA Period 16) Ending Value 1011709.9


(MA Period 17) Ending Value 1011877.6
(MA Period 20) Ending Value 1012708.45
(MA Period 21) Ending Value 1012708.45



(MA Period 19) Ending Value 1011249.25
(MA Period 18) Ending Value 1011361.45


(MA Period 22) Ending Value 1014502.45

(MA Period 25) Ending Value 1012779.7

(MA Period 24) Ending Value 1013549.65

(MA Period 23) Ending Value 1012634.2
(MA Period 27) Ending Value 1012901.5


(MA Period 28) Ending Value 1012311.7
(MA Period 26) Ending Value 1012156.45

(MA Period 29) Ending Value 1011574.9


(MA Period 30) Ending Value 1011574.9

(MA Period 31) Ending Value 1012694.65

(MA Period 33) Ending Value 1013365.6

(MA Period 32) E

[[<backtrader.cerebro.OptReturn at 0x7f56e7be8d30>],

In [12]:
ending = cerebro.broker.getvalue()
print(f'Final Portfolio Value: {ending}')

Final Portfolio Value: 1000000.0


In [13]:
print(f'Returns: {round(((ending - starting)/starting)*100, 2)}%')

Returns: 0.0%


In [14]:
#cerebro.plot()